In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from keras.models import Model,load_model
from keras.utils import np_utils
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Input,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Activation,Dense,Flatten,Add
from IPython.display import SVG

In [2]:
training_data=pd.read_json('train.json')

In [4]:
train_data=np.array(training_data['data'][0:int(training_data.shape[0]*4/5)].tolist())
train_label=np.array(training_data['labels'][0:int(training_data.shape[0]*4/5)].tolist())
val_data=np.array(training_data['data'][int(training_data.shape[0]*4/5):(training_data.shape[0])].tolist())
val_label=np.array(training_data['labels'][int(training_data.shape[0]*4/5):(training_data.shape[0])].tolist())
#split data in to training set and validation set as 4:1

In [6]:
ffin=[]
fin=[]
tmp=[];
ttmp=[];
for i in train_data:
    for k in range(32):
        for j in range(32):
            ttmp.append(i[k*32+j]/255.0)
            ttmp.append(i[k*32+j+1024]/255.0)
            ttmp.append(i[k*32+j+2048]/255.0)
            tmp.append(ttmp)
            ttmp=[]
        fin.append(tmp)
        tmp=[]
    ffin.append(fin)
    fin=[]

train_data=np.array(ffin)

#convert train_data from 1x3072 into 32x32x3

In [7]:
ffin=[]
fin=[]
tmp=[];
ttmp=[];
for i in val_data:
    for k in range(32):
        for j in range(32):
            ttmp.append(i[k*32+j]/255.0)
            ttmp.append(i[k*32+j+1024]/255.0)
            ttmp.append(i[k*32+j+2048]/255.0)
            tmp.append(ttmp)
            ttmp=[]
        fin.append(tmp)
        tmp=[]
    ffin.append(fin)
    fin=[]
    
val_data=np.array(ffin)
ffin=[]

#convert val_data from 1x3072 into 32x32x3

In [9]:
train_label=np_utils.to_categorical(train_label, 2)
val_label=np_utils.to_categorical(val_label, 2)
#convert label into hot code

In [7]:
input_layer=Input((32,32,3))
'''
c0 = Conv2D(16, (3, 3), activation='relu', padding='same') (input_layer)
c0 = Conv2D(16, (3, 3), activation='relu', padding='same') (c0)
p0 = Dropout(0.3) (c0)

c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (p0)
c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (c1)
c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (c1)
p1 = Dropout(0.3) (c1)
p1=MaxPooling2D()(p1)

c2 = Conv2D(32, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(32, (3, 3), activation=None, padding='same') (c2)
rc2 = BatchNormalization()(c2)
rc2 = Activation('relu')(rc2)
rc2 = Conv2D(32, (3, 3), activation='relu', padding='same') (rc2)
rc2 = Conv2D(32, (3, 3), activation='relu', padding='same') (rc2)
rc2 = Add() ([c2,rc2])
p2 = Dropout(0.5) (rc2)

c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(64, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(64, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(64, (3, 3), activation='relu', padding='same') (rc3)
p2 = Dropout(0.5) (rc3)


c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(64, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(64, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(64, (3, 3), activation='relu', padding='same') (rc3)

p2 = Dropout(0.5) (rc3)


c3 = Conv2D(128, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(128, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(128, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(128, (3, 3), activation='relu', padding='same') (rc3)

'''
nerual=16       #64 is worse than above network

fconv=Conv2D(64, (7, 7), strides=(2,2),activation='relu', padding='same') (input_layer)

c0 = Conv2D(nerual, (3, 3), activation='relu', padding='same') (fconv)
c0 = Conv2D(nerual, (3, 3), activation='relu', padding='same') (c0)
p0 = Dropout(0.1) (c0)

c1 = Conv2D(nerual, (3, 3), activation='relu', padding='same') (p0)
c1 = Conv2D(nerual, (3, 3), activation='relu', padding='same') (c1)
c1 = Conv2D(nerual, (3, 3), activation='relu', padding='same') (c1)
p1 = Dropout(0.1) (c1)
p1=MaxPooling2D()(p1)

c2 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(nerual*2, (3, 3), activation=None, padding='same') (c2)
rc2 = BatchNormalization()(c2)
rc2 = Activation('relu')(rc2)
rc2 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (rc2)
rc2 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (rc2)
#rc2 = Add() ([c2,rc2])
p2 = Dropout(0.5) (rc2)

c3 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(nerual*2, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(nerual*2, (3, 3), activation='relu', padding='same') (rc3)
#rc3 = Add() ([c3,rc3])
p2 = Dropout(0.5) (rc3)

c3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(nerual*4, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)
#rc3 = Add() ([c3,rc3])
p2 = Dropout(0.5) (rc3)

c3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(nerual*4, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)
#rc3 = Add() ([c3,rc3])
p2 = Dropout(0.5) (rc3)

c3 = Conv2D(nerual*8, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(nerual*8, (3, 3), activation=None, padding='same') (c3)
rc3 = BatchNormalization()(c3)
rc3 = Activation('relu')(rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)
rc3 = Conv2D(nerual*4, (3, 3), activation='relu', padding='same') (rc3)

flat = Flatten()(rc3)
softmax_layer = Dense(units=2,activation="softmax")(flat)
model=Model(inputs=input_layer,outputs=softmax_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


#Two Network models are tried. The fisrt is simple CNN with dropout. Second is a larger Network with ResNet shortcut.

In [14]:
lr_reduce = ReduceLROnPlateau(factor=0.1, cooldown=0, patience=5, min_lr=0.e-6)
early_stop = EarlyStopping(min_delta=0.001, patience=15)
checkpoint = ModelCheckpoint("./Second_net.model", save_best_only=True, verbose=1)
model.fit(train_data, train_label,batch_size=32,epochs=100,validation_data=(val_data, val_label),callbacks=[lr_reduce, early_stop,checkpoint])


Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 667us/step - loss: 0.6772 - acc: 0.5900 - val_loss: 0.6615 - val_acc: 0.6260

Epoch 00001: val_loss improved from inf to 0.66145, saving model to ./Seconf_net_no_shortcut.model
Epoch 2/100
8000/8000 [==============================] - 3s 402us/step - loss: 0.6209 - acc: 0.6676 - val_loss: 0.6672 - val_acc: 0.5700

Epoch 00002: val_loss did not improve from 0.66145
Epoch 3/100
8000/8000 [==============================] - 3s 433us/step - loss: 0.5856 - acc: 0.7001 - val_loss: 0.7453 - val_acc: 0.6575

Epoch 00003: val_loss did not improve from 0.66145
Epoch 4/100
8000/8000 [==============================] - 3s 435us/step - loss: 0.5575 - acc: 0.7192 - val_loss: 0.5132 - val_acc: 0.7450

Epoch 00004: val_loss improved from 0.66145 to 0.51318, saving model to ./Seconf_net_no_shortcut.model
Epoch 5/100
8000/8000 [==============================] - 3s 429us/step - loss: 0.5320 - acc: 0.7

In [17]:
testing_data=pd.read_json('test.json')
test_data=np.array(testing_data['data'][0:int(testing_data.shape[0]*4/5)].tolist())
ffin=[]
fin=[]
tmp=[];
ttmp=[];
for i in test_data:
    for k in range(32):
        for j in range(32):
            ttmp.append(i[k*32+j]/255.0)
            ttmp.append(i[k*32+j+1024]/255.0)
            ttmp.append(i[k*32+j+2048]/255.0)
            tmp.append(ttmp)
            ttmp=[]
        fin.append(tmp)
        tmp=[]
    ffin.append(fin)
    fin=[]

test_data=np.array(ffin)
ffin=[]
#get test_data in 32x32x3 format

In [19]:
test_label=np.array(testing_data['labels'][0:int(testing_data.shape[0]*4/5)].tolist())
test_label=np_utils.to_categorical(test_label, 2)

In [21]:
prediction=model.predict(test_data)
prediction=np.argmax(prediction,axis=1)

In [22]:

test_label=np.argmax(test_label,axis=1)

In [23]:
ra=0
wa=0
for i in range(len(prediction)):
    if prediction[i]==test_label[i]:
        ra+=1
    else:
        wa += 1

print('Acc:',1.0*ra/(ra+wa))

1417
211
0.8703931203931204
